### TFG-Aplicación de ténicas de Deep Learning para la predicción de demanda en medios de transporte

*Autor: Pedro Jesús Motos Nicolás* 

*Tutor: Fernando Terroso Sáenz*


 *-Importación de todas las funciones y clases que vamos a utilizar más adelante*

In [1]:
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


#### Red neuronal - LSTM 

In [2]:
Archivo_Datos = 'Datos_LSTM.csv'

In [3]:
'''Función para crear un nuevo conjunto de datos. Donde dataX va a ser el numero
de viajes que se han realizado en un momento dado y dataY el numeor de viajes 
en el proximo momento'''
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

'''Corregimos la semilla del numero aleatorio para poder estudiar los datos'''
numpy.random.seed(7)


'''Cargamos el conjunto de datos y convertimos los valores enteros en valores 
de comaflotante que son más adecuados para modelar con una red neuronal'''
dataframe = read_csv(Archivo_Datos, usecols=[1,2], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')

'''Normalizamos la escala de los datos de entrada, reescalamos los datos al
rango de 0 a 1. Para ello utilizamos la clase de preprocesamiento MinMaxScaler 
de la biblioteca scikit-learn.'''
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)


'''Ahora se va a pasar a dividir el conjunto de datos en dos, los datos de prueba (33%)
y los datos de entrenamiento (67%)'''
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]


'''Preparación del conjunto de entrenamiento y de prueba para el modelado'''
look_back = 2
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)


'''La red espera los datos de entrada de la siguiente forma [muestra, pasos, características] 
y debemos preparalos para ello.''' 
trainX = numpy.reshape(trainX, (trainX.shape[0], 2,trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 2,testX.shape[1]))


'''Diseño y adaptación de la red LSTM'''
model = Sequential()
model.add(LSTM(4, input_shape=(2, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

'''Estimación del rendimiento de entrenamiento y de test'''
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)


'''Anteriormente habíamos transformado los datos(en escala de 0 a 1), ahora tenemos que devolverlos
a las mismas unidades de los datos originales, antes de calcular el error'''
trainPredict_dataset_new = numpy.zeros(shape=(len(trainPredict), 2))
trainPredict_dataset_new[:,0] = trainPredict[:,0]
trainPredict = scaler.inverse_transform(trainPredict_dataset_new)[:,0]

trainY_dataset_new = numpy.zeros(shape=(len(trainY), 2))
trainY_dataset_new[:,0] = trainY
trainY = scaler.inverse_transform(trainY_dataset_new)[:,0]

testPredict_dataset_new = numpy.zeros(shape=(len(testPredict), 2))
testPredict_dataset_new[:,0] = testPredict[:,0] 
testPredict = scaler.inverse_transform(testPredict_dataset_new)[:,0]

testY_dataset_new = numpy.zeros(shape=(len(testY), 2))
testY_dataset_new[:,0] = testY
testY = scaler.inverse_transform(testY_dataset_new)[:,0]



'''Calculo de la media del error cuadrático.'''
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
print('Train Score: %.2f RMSE' % (trainScore))

testScore = math.sqrt(mean_squared_error(testY, testPredict))
print('Test Score: %.2f RMSE' % (testScore))





Epoch 1/100
 - 7s - loss: 0.0232
Epoch 2/100
 - 6s - loss: 0.0121
Epoch 3/100
 - 6s - loss: 0.0074
Epoch 4/100
 - 6s - loss: 0.0048
Epoch 5/100
 - 6s - loss: 0.0039
Epoch 6/100
 - 7s - loss: 0.0038
Epoch 7/100
 - 7s - loss: 0.0037
Epoch 8/100
 - 7s - loss: 0.0037
Epoch 9/100
 - 7s - loss: 0.0037
Epoch 10/100
 - 6s - loss: 0.0036
Epoch 11/100
 - 6s - loss: 0.0037
Epoch 12/100
 - 7s - loss: 0.0036
Epoch 13/100
 - 7s - loss: 0.0036
Epoch 14/100
 - 7s - loss: 0.0035
Epoch 15/100
 - 7s - loss: 0.0035
Epoch 16/100
 - 7s - loss: 0.0035
Epoch 17/100
 - 7s - loss: 0.0035
Epoch 18/100
 - 7s - loss: 0.0034
Epoch 19/100
 - 7s - loss: 0.0034
Epoch 20/100
 - 6s - loss: 0.0034
Epoch 21/100
 - 7s - loss: 0.0033
Epoch 22/100
 - 6s - loss: 0.0034
Epoch 23/100
 - 7s - loss: 0.0033
Epoch 24/100
 - 6s - loss: 0.0033
Epoch 25/100
 - 6s - loss: 0.0033
Epoch 26/100
 - 7s - loss: 0.0033
Epoch 27/100
 - 7s - loss: 0.0033
Epoch 28/100
 - 7s - loss: 0.0032
Epoch 29/100
 - 7s - loss: 0.0033
Epoch 30/100
 - 7s - lo